In [1]:
from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session

from database import get_db, init_db
from sync import update_db

from api_functions import get_overview, get_chart_data, get_assets

In [2]:
# initialize the server database
init_db()

INFO:root:Trying to connect to the database...
INFO:root:Successfully connected to the database!
INFO:root:DB tables []
INFO:root:DB views []
INFO:root:Model tables ['BourseHistory', 'CryptocurrencyHistory', 'CurrencyHistory', 'GoldHistory', 'FundHistory']
INFO:root:Model views ['TotalIndex_view', 'WeightedIndex_view', 'Cryptocurrency_view']
INFO:root:Checking table BourseHistory
INFO:root:Table BourseHistory does not exist in the database! Creating table...
INFO:root:Checking table CryptocurrencyHistory
INFO:root:Table CryptocurrencyHistory does not exist in the database! Creating table...
INFO:root:Checking table CurrencyHistory
INFO:root:Table CurrencyHistory does not exist in the database! Creating table...
INFO:root:Checking table GoldHistory
INFO:root:Table GoldHistory does not exist in the database! Creating table...
INFO:root:Checking table FundHistory
INFO:root:Table FundHistory does not exist in the database! Creating table...
INFO:root:Views will be created and replaced


In [3]:
update_db()

INFO:root:Calling source api for index total
INFO:root:Transforming raw data
INFO:root:Data sample: 
      Date_En           Index_Id Index_Type      Value  Previous_Value  \
0 2022-01-01  32097828799138957      total  1378770.0       1378777.4   
1 2022-01-02  32097828799138957      total  1378600.0       1379041.1   
2 2022-01-03  32097828799138957      total  1376040.0       1376049.3   
3 2022-01-04  32097828799138957      total  1362790.0       1362828.6   
4 2022-01-05  32097828799138957      total  1362340.0       1363095.8   

   Closed_Day  
0       False  
1       False  
2       False  
3       False  
4       False  
INFO:root:Inserting to db
INFO:root:Calling source api for index weighted
INFO:root:Transforming raw data
INFO:root:Data sample: 
      Date_En           Index_Id Index_Type     Value  Previous_Value  \
0 2022-01-01  67130298613737946   weighted  363469.0        363469.0   
1 2022-01-02  67130298613737946   weighted  363341.0        363519.0   
2 2022-01-03  67

In [13]:
from models import BourseHistory
records = [BourseHistory(**record) for record in df.to_dict(orient='records')]

In [38]:
records = df.to_dict(orient='records')

In [39]:
from sqlalchemy.dialects.postgresql import insert
# from sqlalchemy import insert
stmt = insert(BourseHistory).values(records)
stmt = stmt.on_conflict_do_update(
    index_elements =['Index_Id', 'Date_En'],
    set_={
        'Index_Type': stmt.excluded.Index_Type,
        'Value': stmt.excluded.Value,
        'Previous_Value': stmt.excluded.Previous_Value,
        'Closed_Day': stmt.excluded.Closed_Day
    }
)

In [37]:
db_gen = get_db()
db = next(db_gen)

In [40]:

db.execute(stmt)

In [41]:
db.commit()

In [42]:
db.close()

In [12]:
records = []
for record_data in df.to_dict(orient='records'):
    # Check if the primary key already exists
    existing_record = db.query(BourseHistory).get(record_data['primary_key'])
    if existing_record:
        # Handle update or skip the record
        # For example, you can update existing_record with record_data
        # existing_record.some_field = record_data['some_field']
        # Or, you can skip the record
        continue
    # If primary key doesn't exist, create a new object
    records.append(BourseHistory(**record_data))

KeyError: 'primary_key'

In [ ]:
existing_record = db.query(BourseHistory).get(record_data['primary_key'])